In [ ]:
import os
import re
import json
from textwrap import dedent
from typing import List, Literal, Tuple, TypedDict

from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage
from langchain_core.output_parsers import JsonOutputParser # JsonOutputParser는 with_structured_output 사용으로 인해 직접 사용되지 않음
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# --- 변경된 부분: Google 관련 임포트 제거하고 Ollama 관련 임포트 추가 ---
# from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
# --- 변경 끝 ---

from langchain_community.vectorstores import Chroma
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import MessagesState
import gradio as gr
import uuid

# 1. 환경 설정
load_dotenv() # .env 파일에서 환경 변수를 로드하지만, GOOGLE_API_KEY는 더 이상 필요 없습니다.

# Google Gemini API 키 설정 부분은 더 이상 필요 없으므로 주석 처리 또는 제거
# os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEY"

# 2. 카페 메뉴 데이터 로드 및 파싱
def load_and_parse_menu_data(file_path: str) -> List[Document]:
    """
    카페 메뉴 데이터를 파일에서 로드하고 파싱하여 LangChain Document 객체 리스트로 반환합니다.
    """
    menu_items = []
    current_menu = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line: # 빈 줄이면 이전 메뉴 아이템 저장 후 초기화
                if current_menu and "name" in current_menu:
                    menu_items.append(
                        Document(
                            page_content=f"이름: {current_menu['name']}\n가격: {current_menu.get('price', '정보 없음')}\n주요 원료: {current_menu.get('ingredients', '정보 없음')}\n설명: {current_menu.get('description', '정보 없음')}",
                            metadata={"menu_name": current_menu["name"]}
                        )
                    )
                current_menu = {}
                continue

            if re.match(r'^\d+\.\s*(.+)', line): # 메뉴 이름 (예: 1. 아메리카노)
                match = re.match(r'^\d+\.\s*(.+)', line)
                current_menu["name"] = match.group(1).strip()
            elif line.startswith('• 가격:'):
                current_menu["price"] = line.replace('• 가격:', '').strip()
            elif line.startswith('• 주요 원료:'):
                current_menu["ingredients"] = line.replace('• 주요 원료:', '').strip()
            elif line.startswith('• 설명:'):
                current_menu["description"] = line.replace('• 설명:', '').strip()
        
        # 마지막 메뉴 아이템 처리
        if current_menu and "name" in current_menu:
            menu_items.append(
                Document(
                    page_content=f"이름: {current_menu['name']}\n가격: {current_menu.get('price', '정보 없음')}\n주요 원료: {current_menu.get('ingredients', '정보 없음')}\n설명: {current_menu.get('description', '정보 없음')}",
                    metadata={"menu_name": current_menu["name"]}
                )
            )
    return menu_items

# 메뉴 데이터 로드 (경로는 이전 답변에서 수정했던 대로 유지)
menu_data_path = "C:\\mylangchain\\langchain_basic\\data\\cafe_menu_data.txt" 
menu_documents = load_and_parse_menu_data(menu_data_path)

# 3. 벡터 DB 구축 (Chroma 사용)
# --- 변경된 부분: GoogleEmbeddings 대신 OllamaEmbeddings 사용 ---
embedding_function = OllamaEmbeddings(model="qwen2.5:1.5b") # Ollama에 다운로드된 모델 이름과 동일하게
# --- 변경 끝 ---

vector_db_path = "./chroma_db_cafe_menu"

# 기존 DB가 있으면 로드, 없으면 새로 생성
if os.path.exists(vector_db_path):
    menu_db = Chroma(persist_directory=vector_db_path, embedding_function=embedding_function)
    print("기존 Chroma DB를 로드했습니다.")
else:
    menu_db = Chroma.from_documents(
        documents=menu_documents,
        embedding=embedding_function,
        persist_directory=vector_db_path
    )
    menu_db.persist()
    print("새로운 Chroma DB를 생성하고 저장했습니다.")

# 4. LangGraph 상태 정의
class AssistantState(MessagesState):
    """
    LangGraph 상태를 정의합니다.
    MessagesState를 상속받아 메시지 이력을 자동으로 관리합니다.
    추가적으로 문의 유형 (query_type)과 검색된 문서 (documents)를 저장합니다.
    """
    query_type: Literal["menu_query", "price_query", "recommendation_request", "other"] = Field(
        default="other",
        description="분류된 사용자 문의 유형 (메뉴, 가격, 추천, 기타)",
    )
    documents: List[Document] = Field(
        default_factory=list,
        description="검색된 관련 문서 목록",
    )

# 5. 모델 정의
# --- 변경된 부분: ChatGoogleGenerativeAI 대신 ChatOllama 사용 ---
llm = ChatOllama(model="qwen2.5:1.5b") # Ollama에 다운로드된 모델 이름과 동일하게
# --- 변경 끝 ---

# 6. 문의 분류 로직 (Query Classifier)
# 6. 문의 분류 로직 (Query Classifier)
class QueryClassifier(BaseModel):
    query_type: Literal["menu_query", "price_query", "recommendation_request", "other"] = Field(
        description="사용자 문의의 분류된 유형입니다. 'menu_query', 'price_query', 'recommendation_request', 'other' 중 하나여야 합니다."
    )
    reasoning: str = Field(
        description="왜 그렇게 분류했는지에 대한 간략한 설명입니다."
    )

# --- 변경된 부분: JsonOutputParser 인스턴스 생성 및 체인 연결 방식 변경 ---
# JsonOutputParser 인스턴스 생성 시 pydantic_object를 직접 전달
parser = JsonOutputParser(pydantic_object=QueryClassifier)

query_classifier_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            dedent(
                """
                당신은 고객의 카페 관련 문의를 분류하는 전문 시스템입니다.
                다음 문의 유형 중 하나로 사용자의 질문을 정확히 분류하세요:
                - `menu_query`: 특정 메뉴에 대한 정보(재료, 맛, 특징 등)를 묻는 경우
                - `price_query`: 특정 메뉴나 전반적인 메뉴의 가격을 묻는 경우
                - `recommendation_request`: 메뉴 추천을 요청하는 경우 (예: '오늘 추천 메뉴는?', '맛있는 커피 추천해줘')
                - `other`: 위 세 가지 유형에 해당하지 않는 일반적인 문의 또는 인사

                응답은 JSON 형식으로만 제공해야 합니다.
                {format_instructions}
                """
            )
        ),
        ("human", "{question}"),
    ]
).partial(
    # JsonOutputParser 객체의 get_format_instructions() 메서드를 사용하여 프롬프트에 형식 지시 포함
    format_instructions=parser.get_format_instructions()
)

# 체인 수정: LLM의 원시 문자열 출력을 JsonOutputParser가 처리하도록 연결
query_classifier_chain = query_classifier_prompt | llm | parser
# --- 변경 끝 ---

def classify_query(state: AssistantState):
    """사용자 문의를 분류하고 상태를 업데이트합니다."""
    print("---분류 중---")
    last_message = state["messages"][-1].content
    try:
        # parser가 LLM의 JSON 문자열 출력을 QueryClassifier 객체로 변환합니다.
        classification_result: QueryClassifier = query_classifier_chain.invoke({"question": last_message})
        query_type = classification_result.query_type
        print(f"분류 결과: {query_type}")
        return {"query_type": query_type}
    except Exception as e: # PydanticValidationError 등 파싱 오류를 포함한 모든 예외 처리
        print(f"문의 분류 중 오류 발생: {e}")
        # LLM이 유효한 JSON을 반환하지 못하거나 파싱 오류가 발생할 경우 'other'로 폴백
        return {"query_type": "other"}

# 7. 응답 생성 함수들
def extract_menu_info(doc: Document) -> dict:
    """Vector DB 문서에서 구조화된 메뉴 정보 추출"""
    content = doc.page_content
    menu_name = doc.metadata.get('menu_name', 'Unknown')
    
    price_match = re.search(r'가격:\s*(₩[\d,]+)', content)
    ingredients_match = re.search(r'주요 원료:\s*(.+?)(?:\n|$)', content)
    description_match = re.search(r'설명:\s*(.+?)(?:\n|$)', content, re.DOTALL)
    
    return {
        "name": menu_name,
        "price": price_match.group(1).strip() if price_match else "가격 정보 없음",
        "ingredients": ingredients_match.group(1).strip() if ingredients_match else "주요 원료 정보 없음",
        "description": description_match.group(1).strip() if description_match else "설명 없음"
    }

def handle_menu_query(state: AssistantState):
    """메뉴 문의에 대한 응답을 생성하고 상태를 업데이트합니다."""
    print("---메뉴 문의 처리 중---")
    user_message = state["messages"][-1].content
    
    # 의미론적 검색 (Semantic Search)
    docs = menu_db.similarity_search(user_message, k=4)
    state["documents"] = docs # 검색된 문서 상태에 저장
    
    if not docs:
        response_content = "죄송합니다. 요청하신 메뉴 정보를 찾을 수 없습니다. 다른 메뉴를 문의해 주시겠어요?"
    else:
        menu_info_list = [extract_menu_info(doc) for doc in docs]
        
        # LLM을 사용하여 추출된 정보를 바탕으로 자연스러운 응답 생성
        menu_info_str = "\n\n".join([
            f"- 이름: {info['name']}\n  가격: {info['price']}\n  주요 원료: {info['ingredients']}\n  설명: {info['description']}"
            for info in menu_info_list
        ])
        
        prompt_template = ChatPromptTemplate.from_messages([
            ("system", dedent(f"""
            당신은 카페 메뉴에 대한 질문에 친절하고 상세하게 답변하는 AI 어시스턴트입니다.
            다음 메뉴 정보를 바탕으로 사용자의 질문에 답하세요.

            ---메뉴 정보---
            {menu_info_str}
            ---------------

            사용자의 질문과 관련된 메뉴 정보를 중심으로 답변을 생성하고, 만약 정보가 부족하다면 없다고 솔직하게 말해주세요.
            """
            )),
            ("human", "{question}")
        ])
        
        chain = prompt_template | llm
        response_content = chain.invoke({"question": user_message}).content
        
    return {"messages": [AIMessage(content=response_content)]}

def handle_price_query(state: AssistantState):
    """가격 문의에 대한 응답을 생성하고 상태를 업데이트합니다."""
    print("---가격 문의 처리 중---")
    user_message = state["messages"][-1].content
    
    # 가격 문의는 좀 더 일반적인 가격 쿼리
    docs = menu_db.similarity_search(user_message + " 가격", k=5) 
    state["documents"] = docs
    
    if not docs:
        response_content = "죄송합니다. 요청하신 메뉴의 가격 정보를 찾을 수 없습니다. 정확한 메뉴 이름을 알려주시면 더 정확히 찾아드릴 수 있습니다."
    else:
        menu_info_list = [extract_menu_info(doc) for doc in docs]
        
        price_info_str = "\n\n".join([
            f"- {info['name']}: {info['price']}"
            for info in menu_info_list
        ])
        
        prompt_template = ChatPromptTemplate.from_messages([
            ("system", dedent(f"""
            당신은 카페 메뉴의 가격에 대해 정확하게 답변하는 AI 어시스턴트입니다.
            다음 가격 정보를 바탕으로 사용자의 질문에 답하세요.

            ---가격 정보---
            {price_info_str}
            ---------------

            사용자의 질문과 관련된 메뉴의 가격을 중심으로 답변을 생성하고, 만약 정보가 부족하다면 없다고 솔직하게 말해주세요.
            간단하고 명확하게 가격만 언급하는 것이 좋습니다.
            """)),
            ("human", "{question}")
        ])
        
        chain = prompt_template | llm
        response_content = chain.invoke({"question": user_message}).content
        
    return {"messages": [AIMessage(content=response_content)]}

def handle_recommendation_request(state: AssistantState):
    """추천 요청에 대한 응답을 생성하고 상태를 업데이트합니다."""
    print("---추천 요청 처리 중---")
    user_message = state["messages"][-1].content
    
    # 추천 요청은 사용자 메시지 + 기본 추천 키워드로 검색
    docs = menu_db.similarity_search(user_message, k=3)
    if not docs: # 사용자 메시지로 적절한 추천이 없으면 '인기 메뉴'로 대체
        docs = menu_db.similarity_search("인기 메뉴", k=3)
    state["documents"] = docs
    
    if not docs:
        response_content = "죄송합니다. 현재 추천해 드릴 메뉴가 없습니다. 어떤 종류의 메뉴를 선호하시는지 알려주시면 더 적합한 추천을 해드릴 수 있습니다."
    else:
        menu_info_list = [extract_menu_info(doc) for doc in docs]
        
        recommendation_info_str = "\n\n".join([
            f"- 이름: {info['name']}\n  가격: {info['price']}\n  설명: {info['description']}"
            for info in menu_info_list
        ])
        
        prompt_template = ChatPromptTemplate.from_messages([
            ("system", dedent(f"""
            당신은 고객의 취향에 맞춰 카페 메뉴를 추천해주는 친절한 AI 어시스턴트입니다.
            다음 메뉴 정보를 바탕으로 사용자에게 적절한 메뉴를 추천해주세요.
            간단한 설명과 가격을 포함하여 추천 이유를 명확하게 제시해주세요.

            ---추천 메뉴 후보---
            {recommendation_info_str}
            --------------------

            추천 메시지는 2~3가지 메뉴를 포함하여 상세하게 설명해주세요.
            """
            )),
            ("human", "{question}")
        ])
        
        chain = prompt_template | llm
        response_content = chain.invoke({"question": user_message}).content
        
    return {"messages": [AIMessage(content=response_content)]}

def handle_other_query(state: AssistantState):
    """기타 문의에 대한 일반적인 응답을 생성하고 상태를 업데이트합니다."""
    print("---기타 문의 처리 중---")
    user_message = state["messages"][-1].content
    
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", dedent("""
        당신은 카페 관련 일반적인 질문에 답변하는 친절한 AI 어시스턴트입니다.
        사용자의 질문에 대해 성의 있고 도움이 되는 답변을 제공하세요.
        메뉴, 가격, 추천과 직접적인 관련이 없는 질문입니다.
        """)),
        ("human", "{question}")
    ])
    
    chain = prompt_template | llm
    response_content = chain.invoke({"question": user_message}).content
    
    return {"messages": [AIMessage(content=response_content)]}

# 8. LangGraph 워크플로우 구성
graph_builder = StateGraph(AssistantState)

# 노드 정의
graph_builder.add_node("classify_query", classify_query)
graph_builder.add_node("handle_menu_query", handle_menu_query)
graph_builder.add_node("handle_price_query", handle_price_query)
graph_builder.add_node("handle_recommendation_request", handle_recommendation_request)
graph_builder.add_node("handle_other_query", handle_other_query)

# 시작점 설정
graph_builder.set_entry_point("classify_query")

# 조건부 라우팅
graph_builder.add_conditional_edges(
    "classify_query",
    lambda state: state["query_type"], # classify_query 노드의 결과인 query_type 값에 따라 라우팅
    {
        "menu_query": "handle_menu_query",
        "price_query": "handle_price_query",
        "recommendation_request": "handle_recommendation_request",
        "other": "handle_other_query",
    },
)

# 모든 처리 노드에서 END로 연결
graph_builder.add_edge("handle_menu_query", END)
graph_builder.add_edge("handle_price_query", END)
graph_builder.add_edge("handle_recommendation_request", END)
graph_builder.add_edge("handle_other_query", END)

# 그래프 컴파일
app = graph_builder.compile()

# 9. Gradio 인터페이스
class ChatBot:
    def __init__(self):
        # 각 스레드(대화 세션)별로 고유한 스레드 ID를 할당하여 대화 이력을 관리
        self.thread_id = str(uuid.uuid4())
        # self.messages_history = {} # MessagesState가 메시지 이력을 관리하므로 이 부분은 더 이상 직접 사용되지 않음

    def chat(self, message: str, history: List[Tuple[str, str]]) -> str:
        print(f"Thread ID: {self.thread_id}")

        # Gradio history를 LangChain BaseMessage로 변환
        lc_history = []
        for human_msg, ai_msg in history:
            if human_msg:
                lc_history.append(HumanMessage(content=human_msg))
            if ai_msg:
                lc_history.append(AIMessage(content=ai_msg))
        
        # 현재 사용자 메시지 추가
        lc_history.append(HumanMessage(content=message))

        # LangGraph 호출
        # input 형식은 state 딕셔너리로 제공
        try:
            # LangGraph는 MessagesState의 `messages` 필드를 자동으로 업데이트하므로,
            # 초기 상태에 `messages`만 넣어주면 됩니다.
            initial_state = {"messages": lc_history}
            final_state = app.invoke(initial_state)
            
            # 최종 AIMessage 반환
            return final_state["messages"][-1].content
            
        except Exception as e:
            print(f"Error occurred during LangGraph invocation: {str(e)}")
            return "죄송합니다. 응답을 생성하는 동안 오류가 발생했습니다. 다시 시도해 주세요."

chatbot = ChatBot()

example_questions = [
    ["아메리카노 설명해 줘"],
    ["카페라떼 가격 얼마야?"],
    ["오늘의 추천 메뉴 있어?"],
    ["카푸치노 재료는?"],
    ["가장 비싼 음료는 뭐야?"],
    ["안녕하세요!"],
]

demo = gr.ChatInterface(
    fn=chatbot.chat,
    title="카페 메뉴 AI 어시스턴트 (Ollama Qwen2)",
    description="메뉴 정보, 가격, 추천 등 카페 관련 질문에 답변해 드립니다.",
    examples=example_questions,
    theme=gr.themes.Soft()
)

# Gradio 앱 실행
if __name__ == "__main__":
    demo.launch()

c:\Users\qwer\AppData\Local\pypoetry\Cache\virtualenvs\langchain-basic-kGdHTiMZ-py3.12\Lib\site-packages\gradio\chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


기존 Chroma DB를 로드했습니다.
* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


Thread ID: 59528836-89ea-47a3-8c40-9b558caf6522
---분류 중---
문의 분류 중 오류 발생: 'dict' object has no attribute 'query_type'
---기타 문의 처리 중---
